In [76]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import pymaster as nmt
import fitsio
import tqdm

In [25]:
def MSE(d1,d2,weight=1):
    return np.sum(np.real(weight*(d1-d2)*np.conjugate(d1-d2)))/np.size(d1)

In [5]:
dat = fitsio.read('/global/cscratch1/sd/zhzhuoqi/surf/run_sysnet/inputs/eBOSS.ELG.NGC.DR7.table.fits')
inds = dat['hpix']

In [18]:
mask = np.zeros(hp.nside2npix(nside))
mask[inds] = 1
mask = mask.astype(np.bool_)
mask_s = nmt.mask_apodization(mask, 1, apotype="C2")

In [7]:
nside = 256
lmax = 3*nside

cl_camb = np.load('cl_camb_nonlin_Y1.npy',allow_pickle=True).item()
ell = np.arange(0,lmax)
s = cl_camb['W1xW1'][0:lmax]*2*np.pi/(ell*(ell+1))
s[0] = 0

t0 = np.ones_like(ell)*2e-6
t0[:2] = 0
t1 = 5e-5/(ell+1)
t1[:2] = 0
t2 = 5e-4/(ell+1)**2
t2[:2] = 0

/tmp/ipykernel_52670/2443363870.py:6: RuntimeWarning: invalid value encountered in true_divide
  s = cl_camb['W1xW1'][0:lmax]*2*np.pi/(ell*(ell+1))


In [109]:
smap = hp.synfast(s,nside)
smap[~mask] = 0
tmap1 = hp.synfast(t0,nside)
tmap1[~mask] = 0
tmap2 = hp.synfast(t0,nside)
tmap2[~mask] = 0
tmap3 = hp.synfast(t0,nside)
tmap3[~mask] = 0

In [142]:
def MP(dmap,t_arr):
    b = nmt.NmtBin.from_nside_linear(nside, 4)
    ell_eff = b.get_effective_ells()
    n = len(t_arr)
    sigma_td = np.zeros((n,1))
    sigma_tt = np.zeros((n,n))

    f_d = nmt.NmtField(mask, [dmap])
    for i in range(len(t_arr)):
        for j in range(0,i+1):
            f_t1 = nmt.NmtField(mask, [t_arr[i]])
            f_t2 = nmt.NmtField(mask, [t_arr[j]])
            cl_tt = nmt.compute_full_master(f_t1, f_t2, b)
            sigma_tt[i,j] = np.sum((2*ell_eff+1)*cl_tt)
            sigma_tt[j,i] = np.sum((2*ell_eff+1)*cl_tt)
        f_t = nmt.NmtField(mask, [t_arr[i]])
        cl_td = nmt.compute_full_master(f_t, f_d, b) 
        sigma_td[i,0] = np.sum((2*ell_eff+1)*cl_td)
    alpha = np.dot(np.linalg.inv(sigma_tt),sigma_td)
    return alpha.reshape(-1)

In [ ]:
mses = []
for _ in tqdm.trange(10):
    np.random.seed(_)
    smap = hp.synfast(s,nside)
    smap[~mask] = 0
    alpha1 = MP(smap,tmap1)
    alpha2 = MP(smap,tmap2)
    shat1 = smap - alpha1*tmap1
    shat2 = smap - alpha2*tmap2
    mses.append(MSE(shat1,shat2))

In [116]:
mses

[6.488092098303125e-07,
 1.0939641740596981e-06,
 4.165520148260136e-07,
 1.096276351035692e-07,
 5.492010520345762e-07,
 3.0518341654372425e-07,
 5.463031491445044e-07,
 6.970191120377409e-07,
 3.0550675211365955e-07,
 1.1163701837788113e-07]

In [157]:
dmap = smap+1.*tmap1+1.*tmap2+1.*tmap3
t_arr = np.array([tmap1,tmap2,tmap3])
alpha1 = MP(dmap,t_arr)
shat1 = dmap - np.dot(t_arr.T,alpha1)
print(MSE(shat1,smap))

6.524037008058986e-07


In [155]:
np.std(tmap1+tmap2+tmap3)/np.std(smap)

1.6895383182126826

In [144]:
dmap = smap+tmap2
alpha2 = MP(dmap,[tmap2])
shat2 = dmap - alpha2*tmap2
print(MSE(shat2,smap))

2.514067434411412e-07


In [145]:
MSE(shat1,shat2)

2.8898083449106918e-06

In [102]:
np.std(0.02*tmap3[mask])

0.0061284082685368665

In [103]:
np.std(dmap[mask])

0.4372748441747765

In [105]:
0.0061284082685368665/0.32312919353297087

0.018965814266210978

In [101]:
mses

[3.668763219889041e-06,
 5.590627758646892e-06,
 6.736967031757072e-06,
 2.571422162469559e-07,
 8.84840216652451e-07,
 1.7662450285308214e-06,
 3.958032050403614e-06,
 2.716880304239235e-07,
 3.9683204994917396e-07,
 3.309471660304734e-07]